# [PyTorch - Learning the Basics](https://pytorch.org/tutorials/beginner/basics/intro.html)

Now, for part four, let's look into building a neural network.

## Build the neural network

Foundationally, neural networks comprise of layers/modules that perform operations on data. The [`torch.nn`](https://pytorch.org/docs/stable/nn.html) namespace provides all the building blocks needed to build a neural network. Every module in PyTorch subclasses the [`nn.Module`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html). A neural network is a module itself that consists of other modules, or layers. This nested structure allows for building and managing complex architectures easily.

Along this notebook, we'll build a neural network to classify images in the FashionMNIST dataset.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

### Get device for training

Ideally, we want to train our model on an accelerator such as CUDA, MPS (M1 Mac, what I'll mostly be using), MTIA, or XPU. If it's available we'll use it, otherwise we'll use the CPU.

In [2]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using mps device


### Define the class

We'll use `nn.Module` to define our neural network and initialize the layers in `__init__`. Every `nn.Module` subclass implements the operations on input data in the `forward` method.

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

Next, let's create an instance of `NeuralNetwork`, move it to the device, and print its structure.

In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


We'll pass the input data through it to actually use the model. This will execute the model's `foward`, along with some other [background operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866). **Do not call `model.forward()` directly!**

Calling the model on the input returns a 2-dimensional tensor with `dim=0` corresponding to each output of 10 raw predicted values for each class, and `dim=1` corresponding to the individual values of each output. We get the prediction probabilities by passing it through an instance of the `nn.Softmax` module.

In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)

print(f"Predicted class: {y_pred}")

Predicted class: tensor([9], device='mps:0')


### Model layers

Let's take a sample minibatch of 3 images of size 28x28 to see what happens to it as we pass it through the network to illustrate what's going on to break down the layers in the FashionMNIST model.

In [7]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


### `nn.Flatten`

We initialize the [`nn.Flatten`](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html) layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values (the minimatch dimension (at `dim=0`) is maintained).

In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)

print(flat_image.size())

torch.Size([3, 784])


### `nn.Linear`

The [linear layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) is a module that applies a linear transformation on the input using its stored weights and biases.

In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)

print(hidden1.size())

torch.Size([3, 20])


### `nn.ReLU`

Non-linear activations are what create the complex mappings between the model's inputs and outputs. They are applied after linear transformations to introduce *nonlinearity*, which helps neural networks learn a wide variety of phenomena.

In this model, we use [`nn.ReLU`](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html) between our linear layers, but there's other activations to introduce non-linearity in your model.

> **！Personal Note**
>
> As far as I can tell, `hidden1`, before `ReLU`, uses `Addmm` and maps each element of the tensor between -1 and 1. After `ReLU`, they're mapped between 0 and 1, where anything that was previously below 0 is just set to 0.

In [10]:
print(f"Before ReLU: {hidden1}\n\n")

hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.2143,  0.3532, -0.1648, -0.0441, -0.0677,  0.1451, -0.1018,  0.1009,
         -0.0349,  0.2999, -0.6760, -0.3985,  0.0988, -0.0711,  0.6325, -0.4741,
          0.0234,  0.1186,  0.6427, -0.0968],
        [ 0.4606,  0.1896,  0.0837,  0.1411,  0.3766,  0.0901, -0.1442, -0.1908,
          0.2644,  0.3386, -0.2272, -0.1684,  0.0178,  0.0539,  0.4104, -0.7913,
         -0.0780, -0.3408,  0.4916, -0.2467],
        [-0.2033,  0.1224, -0.2392, -0.3626,  0.0704,  0.1299, -0.1785,  0.0674,
          0.3826,  0.7751, -0.4771, -0.1844,  0.0345,  0.2057,  0.3365, -0.4669,
         -0.1711, -0.0764,  0.3533,  0.0260]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.3532, 0.0000, 0.0000, 0.0000, 0.1451, 0.0000, 0.1009, 0.0000,
         0.2999, 0.0000, 0.0000, 0.0988, 0.0000, 0.6325, 0.0000, 0.0234, 0.1186,
         0.6427, 0.0000],
        [0.4606, 0.1896, 0.0837, 0.1411, 0.3766, 0.0901, 0.0000, 0.0000, 0.2644,
         0.3386, 0.0000, 0.0000, 0.0178, 0.0539, 0.41

### `nn.Sequential`

[`nn.Sequential`](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) is an ordered container of modules. The data is passed through all the modules in the same order as defined. You can use sequential containers to put together a quick network like `seq_modules` below.

In [11]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)

input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

### `nn.Softmax`

The last linear layer of the neural network returns *logits* - raw values in `[-infty, infty]` - which are passed to the [`nn.Softmax`](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html) module. The logits are scaled to values `[0,1]` representing the model's predicted probabilities for each class. The `dim` parameter indicates the dimension along which the values must sum to 1.

In [17]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

print(torch.sum(pred_probab[0]))

tensor(1., grad_fn=<SumBackward0>)


### Model parameters

Many layers inside a neural network are *parameterized*, in other words, they have associated weights and biases that are optimized during training. Subclassing `nn.Module` automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model's `parameters()` or `named_parameters()` methods.

In the below example, we iterate over each parameter and print its size and a preview of its values.

In [18]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[ 0.0290,  0.0022, -0.0082,  ..., -0.0165, -0.0073, -0.0086],
        [ 0.0241, -0.0324, -0.0098,  ...,  0.0249, -0.0281,  0.0195]],
       device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([-0.0215, -0.0112], device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[ 0.0075,  0.0252, -0.0365,  ..., -0.0302,  0.0387,  0.0167],
        [-0.0204, -0.0100,  0.0141,  ..., -0.0053,  0.0187,  0.0251]],
       device='mps:0', grad_fn=<SliceBac